In [ ]:
Ex_GetProxy

In [ ]:
Proxy Server(代理伺服器)
若每個人都要到同個網站讀取資料，則每次就要重新讀取，對頻寬來說是多麼的浪費，而且用起來非常非常的慢。
假始今天我們有一台 Proxy Server，而區域內所有 WWW 使用者都有設定這台 Proxy Server，那麼情況就改變不同了。
第一個要讀取的使用者會先到這台 Proxy Server 上先找看看有沒有要的資料，
如果沒有，則由這台 Proxy Server 幫他代理到網站抓取一份資料之後放在 Proxy Server 上也傳送給第一位使用者。
而第二位以後的使用者，當他要連到該網站時時，他的機器會先到 Proxy Server 上看看，結果有資料，則就直接由 Proxy Server 取回資料即可，
他所用的速度是校內的網路速度，至少也有 10 Mbps， 絕對比連校外的速度快。

因此設 Proxy 有什麼好處？我相信你一定很明白的了解，就是『快』

In [ ]:
如何建構代理IP的清單(Proxy Pool)
當我們在進行網路爬蟲時，或多或少都會碰到網站設置的反爬蟲機制，其中簡單的是檢查headers(瀏覽器參數)，
複雜些的則會「鎖定大量訪問的IP」、「圖形驗證碼」、「登入」…等等，不過既然有「反爬蟲機制」，當然也就會有「反反爬蟲機制」囉！

今天我要介紹的就是如何建構代理IP的清單( Proxy Pool )，透過代理IP來爬資料我們就可以避開「鎖定大量訪問的IP」的反爬蟲機制！

In [ ]:
尋找提供免費的代理IP的網站
網路上提供免費代理IP的網站有很多，我們簡單在 Google 上搜索「免費代理IP」後就可以發現許多網站了，
我找到的網站並經過測試還不錯的網站分別是 FREE-PROXY 與 US-PROXY

提供一個使用心得是，在使用時要盡量避免使用位在中國的代理IP，會這樣建議的原因是大陸有防火牆的限制，
訪問國外的網站時容易被阻擋或比較不方便，為了避免這些問題，建議直接找美國的代理IP來使用！
- http://free-proxy.cz/zh/proxylist/country/US/https/ping/all
- https://www.us-proxy.org/

In [ ]:
在網站的畫面中我們會看到表格中有許多的 IP地址 和 通道(Port)，稍後我們就是要抓取表格中的這兩個資訊！

In [ ]:
抓取代理IP清單
在這邊測試的網站是 FREE-PROXY，考量讓大家能方便執行這個程式

In [ ]:
# 安裝與載入套件

In [ ]:
# install chromium, its driver, and selenium
!apt-get update
!apt install chromium-chromedriver
!pip install selenium

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Ign:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Get:3 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran35/ InRelease [3,626 B]
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:5 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease [21.3 kB]
Ign:6 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:7 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Get:10 http://security.ubuntu.com/ubuntu bionic-security/main amd64 Packages [817 kB]
Get:11 http://ppa.launchpad.net/marutter/c2d4u3.5/ubuntu bionic InRelease [15.4 kB]
Get:12 http://security.ubuntu.com/ubuntu bionic-security/restric

In [1]:
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import pandas as pd
import re

In [2]:
#設置Selenium參數
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
options.add_argument("--disable-notifications")
driver = webdriver.Chrome(options=options)

In [ ]:
# 收集IP清單
在此先用迴圈的方式讀取1-5個分頁表格中的IP
由於IP的格式是由四組1-3個數字組合而成，並於中間用「.」隔開，因此可以透過正則表達式來抓取資料

In [36]:
IPPool = []
for i in range(1,5):
    url = 'http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/{}'.format(i)
    print('Dealing with {}'.format(url))
    driver.get(url)
    soup = BeautifulSoup(driver.page_source)
    for j in soup.select('tbody > tr'):
        if re.findall('[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{2,3}', str(j)):
            IP = re.findall('[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}\.[0-9]{1,3}', str(j))[0]
            Port = re.findall('class="fport" style="">(.*?)</span>', str(j))[0]
            IPPool.append(pd.DataFrame([{'IP':IP, 'Port':Port}]))
    print('There are {} IPs in Pool'.format(len(IPPool)))
IPPool = pd.concat(IPPool, ignore_index=True)
IPPool

Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/1
There are 35 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/2
There are 70 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/3
There are 105 IPs in Pool
Dealing with http://free-proxy.cz/zh/proxylist/country/US/https/ping/all/4
There are 140 IPs in Pool


,IP,Port
0,209.246.143.213,8080
1,155.138.146.210,8080
2,155.138.159.208,8080
3,167.99.230.151,3128
4,45.77.76.254,8080
...,...,...
135,167.71.249.181,8888
136,35.240.252.181,3128
137,107.173.177.133,5836
138,173.82.17.190,5836


In [ ]:
測試代理IP的有效性
雖然看起來我們好像拿到 104 個 IP 了，但別高興得太早，因為當中絕大多數的IP都無法使用!!(畢竟來源是免費的xDD

因此我們在這裡需要再多一個步驟用來確認這些IP的有效性，而測試的方式就是我們在 request 的時候透過這些IP和Port來訪問網站，當我們訪問成功後我們才把IP保留下來，反之則丟掉

In [37]:
ActIps = []
for IP, Port in zip(IPPool['IP'],IPPool['Port']):
    proxy = {'http':'http://'+ IP + ':' + Port,
             'https':'https://'+ IP + ':' + Port} 
    try:
        # 隨機找的一篇新聞
        url = 'https://www.chinatimes.com/realtimenews/20200205004069-260408'
        resp = requests.get(url, proxies=proxy, timeout=2)
        if str(resp.status_code) == '200':
            ActIps.append(pd.DataFrame([{'IP':IP, 'Port':Port}]))
            print('Succed: {}:{}'.format(IP, Port))
        else:
            print('Failed: {}:{}'.format(IP, Port))
    except:
            print('Failed: {}:{}'.format(IP, Port))
ActIps = pd.concat(ActIps, ignore_index=True)
ActIps

Failed: 209.246.143.213:8080
Failed: 155.138.146.210:8080
Failed: 155.138.159.208:8080
Failed: 167.99.230.151:3128
Failed: 45.77.76.254:8080
Failed: 207.246.89.154:8080
Failed: 155.138.142.213:8080
Failed: 45.77.157.11:8080
Failed: 149.28.50.175:8080
Failed: 64.227.5.88:3128
Failed: 155.138.150.235:8080
Failed: 159.89.234.222:3128
Failed: 137.220.52.247:8080
Failed: 161.35.107.214:3128
Succed: 68.183.121.227:3128
Failed: 161.35.60.216:3128
Failed: 165.227.201.107:3128
Failed: 68.183.49.220:3128
Failed: 161.35.112.151:3128
Failed: 4.14.219.157:3128
Failed: 64.154.38.221:8080
Failed: 207.246.81.79:8080
Failed: 163.153.220.170:8080
Failed: 52.179.18.244:8080
Failed: 45.63.14.189:8080
Failed: 161.35.3.136:3128
Failed: 3.19.238.16:3128
Failed: 155.138.159.243:8080
Failed: 209.222.10.122:8080
Failed: 137.220.53.253:8080
Failed: 140.82.3.127:8080
Failed: 52.191.103.11:3128
Succed: 104.129.202.146:9480
Succed: 104.129.202.144:9480
Succed: 104.129.202.148:10360
Failed: 35.230.21.108:80
Failed: 

,IP,Port
0,68.183.121.227,3128
1,104.129.202.146,9480
2,104.129.202.144,9480
3,104.129.202.148,10360
4,173.82.62.22,5836
5,70.169.150.183,48678
6,198.98.58.178,8080


In [ ]:
執行完會看到如下畫面，如畫面中的內容，大多數都是無效的IP，最後我們就從 104 個 IP 中找到 6 個有效的 IP 囉!

In [38]:
ActIps

,IP,Port
0,68.183.121.227,3128
1,104.129.202.146,9480
2,104.129.202.144,9480
3,104.129.202.148,10360
4,173.82.62.22,5836
5,70.169.150.183,48678
6,198.98.58.178,8080
